# HOMEWORK 1
This is my first Project which was given to me as an exercise for the Theoretical Astrophysics II: Polarized Radiative Transfer course at the university of Freiburg. The goal is to synthesize a spectral line profile in Local Thermodynamic Equilibrium from a given atmospheric model. 

To do this I am going to calculate level populations using the Boltzmann and Saha equations, compute line opacity, source function and solve the radiative transfer equation.